# Consideraciones para el entrenamiento

En el punto 5 del paper hablan de cómo han entrenado al transformer, así que vamos a verlo

## Datos

Para entrenar al transformer dicen que han usado el dataset [WMT 2014 English-German](https://aclanthology.org/W14-3302.pdf), pero como hemos instalado la librería de transformers de huggingface, vamos a usar su librería datasets para usar el dataset [opus100](https://huggingface.co/datasets/opus100) del inglés al español (`en-es`)

## Hardware

Han usado 8 Nvidia P100. Supongo que ninguno de nosotros tenemos acceso a 8 GPUs de ese tipo, por lo que haremos un código que sea ejecutable en [Colab](https://colab.research.google.com/), que al día de escribir este curso ofrece GPUs como la Nvidia T4 que tiene la misma VRAM que la P100, pero con una arquitectura más nueva. Por lo que en vez de usar 8 GPUs como las del paper, usaremos una un poco mejor que una de las que usaron ellos

## Steps

Dicen que entrenaron el modelo 100.000 pasos o 12 horas. Colab ofrece 12 horas de GPU, así que hacermos algo similar, entrenaremos unos 100.000 steps o cuando estemos un poco por debajo de las 12 horas pararemos el entrenamiento

## Optimizador

Los investigadores usaron un optimizador Adam con $\beta_1 = 0.9$, $\beta_2 = 0.98$ y $\epsilon = 10^{-9}$ y que variaron el learning rate según la fórmula

$$lr = d_{model}^{-0.5} \cdot min \left(stepNumber^{-0.5}, stepNumber \cdot stepWarmUp^{-1.5} \right)$$

Esto corresponde a un aumento lineal de la tasa de aprendizaje para los primeros pasos de entrenamiento (pasos de calentamiento (warmup)), y a una disminución posterior proporcional a la raíz cuadrada inversa del número de pasos. Utilizaron 4000 pasos de calentamiento

### Implementación del optimizador

Implementar el optimizador es sencillo, ya que como hemos visto ya que la función `torch.optim.Adam` hace todo, así que lo que tenemos que hacer es

```python
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
```

### Implementación del scheduler del lr

Para hacer esto vamos a usar la función de Pytorch `torch.optim.lr_scheduler.LambdaLR`, que añade un scheduler al learning rate. Lo vamos a implementar de la siguiente manera

Primero creamos una función que calcula el valor que debe tener el learning rate en función de la época en la que está, es decir, creamos una función que resuelva la fórmula que indican en el paper

```python
def calculate_lr(step_num, dim_embeding_model=512, warmup_steps=4000):
    step_num += 1e-7 # Avoid division by zero
    return (dim_embeding_model**-0.5) * min(step_num**-0.5, step_num*(warmup_steps**-1.5))
```

Se puede ver que hemos añadido `step_num += 1e-7` porque si no en la época 0 intenta hacer `step_num**-0.5` lo cual daría error porque no se puede elevar 0 a un número negativo

A continuación añadimos el scheduler del learning rate al optimizador que hemos creado antes

```python
lr_lambda = lambda step: calculate_lr(step, dim_embeding_model=dim_embedding)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
```

Por último, en el bucle de entrenamiento, después de hacer `optimizer.step()`, hay que además hacer `scheduler.step()` para modificar el valor del learning rate

## Regularización

Durante el entrenamiento utilizaron dropout con una probabilidad de 0.1 después de cada capa de atención y cada capa feed-forward. Además utilizaron label smoothing

El dropout ya lo hemos explicado, pero hacemos un recordatorio rápido. El dropout consiste en durante el entrenamiento desactivar la conexión entre neuronas aleatoriamente y con una probabilidad.

El label smoothing es un suavizado de las probabilidades de la secuencia de salida. Supongamos que le metemos al transformer la frase en español `¿Cuál es tu nombre?` y el trasnformer ha generado `What is your` de modo que la siguiente palabra tiene que ser `name`. Por lo que de todos los posibles tokens del inglés, el correspondiente a `name` debería tener la etiqueta de 1 y el resto de 0. Pero el label smoothing lo que hace es que name tenga una etiqueta de por ejemplo 0.95 y el resto 0.000001

Se define un valor de $1-\epsilon$, y se asigna la etiqueta del token que toca a $1-\epsilon$ y el resto de tokens tienen la etiqueta de $\epsilon / \left(numTokens-1 \right)$

### Implementación del dropout

Creamos una clase para el dropout

In [ ]:
import torch

class Dropout(torch.nn.Module):
    def __init__(self, p=0.1):
        """
        Args:
            p (float): probability of an element to be zeroed. Default: 0.1
        """
        super().__init__()
        self.p = p

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): input tensor

        Returns:
            torch.Tensor: a tensor with the same shape of `x`
        """
        if self.training:
            return torch.nn.functional.dropout(x, p=self.p)
        else:
            return x

### Implementación del label smoothing

Primero escribimos la clase label smoothing y luego la explicamos

In [ ]:
import torch
import torch.nn as nn

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1, dim=-1, ignore_index=-100):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.classes = classes
        self.dim = dim
        self.ignore_index = ignore_index

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.classes - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            mask = (target == self.ignore_index).unsqueeze(1).expand_as(true_dist)
            if mask.any():
                true_dist[mask] = 0
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

Lo que la clase va a hacer es:
 * En la línea `pred = pred.log_softmax(dim=self.dim)` se calcula la función `log_softmax` sobre las predicciones del transformer
 * En `true_dist = torch.zeros_like(pred)` se crea una matriz de ceros del mismo tamaño que las predicciones
 * En `true_dist.fill_(self.smoothing / (self.classes - 1))` ahora se llena esa matriz de ceros con el valor suavizado dividido entre el número de clases menos una. Esto es lo que se aplicará a las etiquetas incorrectas, en vez de un 0 se le aplicará un valor muy pequeño
 * En `true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)` en las posiciones en las que debería haber un 1 se le asigna el valor de confianza, que se calcula como `1-smoothing`
 * En `mask = (target == self.ignore_index).unsqueeze(1).expand_as(true_dist)` creamos una máscara del mismo tamaño que `true_dist` pero en la que todos los elementos en los que `target` sean iguales a `self.ignore_index` valgan 1
 * En `if mask.any(): true_dist[mask] = 0` si hay al menos un 1 en la máscara ponemos a 0 esa posición en `true_dist`
 * Y en `return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))` se calcula la suma de `-true_dist*pred` a lo largo de la dirección dada por `self.dim` y a continuación la media.

Esto es básicamente el cálculo de entropía cruzada o la función de Pytorch `nn.CrossEntropyLoss`, pero con label smoothing

Esta es la función que usaremos para la loss, si no la hubiésemos implementado usaríamos `nn.CrossEntropyLoss` como función de pérdida

## Transformer

Volvemos a escribir cómo quedaría el transformer con el dropout, que es lo único que se añade al modelo

Primero escribimos todas las clases de bajo nivel

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class Dropout(torch.nn.Module):
    def __init__(self, p=0.1):
        """
        Args:
            p (float): probability of an element to be zeroed. Default: 0.1
        """
        super().__init__()
        self.p = p

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): input tensor

        Returns:
            torch.Tensor: a tensor with the same shape of `x`
        """
        if self.training:
            return torch.nn.functional.dropout(x, p=self.p)
        else:
            return x

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        """
        Args:
            vocab_size: size of vocabulary
            embed_dim: dimension of embeddings
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            out: embedding vector
        """
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_len, embedding_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super().__init__()
        self.embedding_dim = embedding_model_dim

        # create constant 'positional_encoding' matrix with values dependant on pos and i
        positional_encoding = torch.zeros(max_sequence_len, self.embedding_dim)
        for pos in range(max_sequence_len):
            for i in range(0, self.embedding_dim, 2):
                positional_encoding[pos, i]     = math.sin(pos / (10000 ** ((2 *     i) / self.embedding_dim)))
                positional_encoding[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i+1)) / self.embedding_dim)))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('positional_encoding', positional_encoding)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """
        # make embeddings relatively larger
        x = x * math.sqrt(self.embedding_dim)
        
        # add encoding matrix to embedding (x)
        sequence_len = x.size(1)
        # x = x + torch.autograd.Variable(self.positional_encoding[:,:sequence_len], requires_grad=False)
        x = x + self.positional_encoding[:,:sequence_len]
        return x

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, query, key, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / math.sqrt(self.dim_embedding)
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.nn.functional.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q: query vector
            K: key vector
            V: value vector
            mask: mask matrix (optional)

        Returns:
            output vector from multi-head attention
        """
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V, mask=mask)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x

class Linear(nn.Module):
    def __init__(self, dim_embedding, vocab_size):
        super().__init__()
        self.linear = nn.Linear(dim_embedding, vocab_size)
        
    def forward(self, x):
        x = self.linear(x)
        return x

class Softmax(nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.softmax(x)
        return x


Escribimos las clases de medio nivel

In [10]:
class EncoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding, prob_dropout=0.1):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.dropout_1 = Dropout(prob_dropout)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
        self.dropout_2 = Dropout(prob_dropout)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        multi_head_attention = self.multi_head_attention(x, x, x)
        dropout1 = self.dropout_1(multi_head_attention)
        add_and_norm_1 = self.add_and_norm_1(x, dropout1)
        feed_forward = self.feed_forward(add_and_norm_1)
        dropout2 = self.dropout_2(feed_forward)
        add_and_norm_2 = self.add_and_norm_2(add_and_norm_1, dropout2)
        return add_and_norm_2

class Encoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx, prob_dropout=0.1):
        super().__init__()
        self.encoder_layers = nn.ModuleList([EncoderLayer(heads, dim_embedding, prob_dropout) for _ in range(Nx)])
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, dim_embedding, max_sequence_len, heads, Nx, prob_dropout=0.1):
        super().__init__()
        self.input_embedding = Embedding(vocab_size, dim_embedding)
        self.positional_encoding = PositionalEncoding(max_sequence_len, dim_embedding)
        self.encoder = Encoder(heads, dim_embedding, Nx, prob_dropout)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        input_embedding = self.input_embedding(x)
        positional_encoding = self.positional_encoding(input_embedding)
        encoder = self.encoder(positional_encoding)
        return encoder

class DecoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding, prob_dropout=0.1):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.masked_multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.dropout_1 = Dropout(prob_dropout)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.encoder_decoder_multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.dropout_2 = Dropout(prob_dropout)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
        self.dropout_3 = Dropout(prob_dropout)
        self.add_and_norm_3 = AddAndNorm(dim_embedding)
    
    def forward(self, x, encoder_output, mask=None):
        """
        Args:
            x: input vector
            encoder_output: output vector from encoder
            mask: mask matrix (optional)
        
        Returns:
            output vector from decoder layer
        """
        Q = x
        K = x
        V = x
        masked_multi_head_attention = self.masked_multi_head_attention(Q, K, V, mask=mask)
        dropout1 = self.dropout_1(masked_multi_head_attention)
        add_and_norm_1 = self.add_and_norm_1(dropout1, x)

        Q = add_and_norm_1
        K = encoder_output
        V = encoder_output
        encoder_decoder_multi_head_attention = self.encoder_decoder_multi_head_attention(Q, K, V)
        dropout2 = self.dropout_2(encoder_decoder_multi_head_attention)
        add_and_norm_2 = self.add_and_norm_2(dropout2, add_and_norm_1)

        feed_forward = self.feed_forward(add_and_norm_2)
        dropout3 = self.dropout_3(feed_forward)
        add_and_norm_3 = self.add_and_norm_3(dropout3, add_and_norm_2)

        return add_and_norm_3

class Decoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx, prob_dropout=0.1):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
            Nx: number of decoder layers
        """
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(heads, dim_embedding, prob_dropout) for _ in range(Nx)])
    
    def forward(self, x, encoder_output, mask=None):
        """
        Args:
            x: input vector
            encoder_output: output vector from encoder
            mask: mask matrix (optional)
        
        Returns:
            output vector from decoder
        """
        for decoder_layer in self.layers:
            x = decoder_layer(x, encoder_output, mask)
        return x

class TransformerDecoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx, vocab_size, max_sequence_len, prob_dropout=0.1):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
            Nx: number of decoder layers
            vocab_size: size of vocabulary
            max_sequence_len: maximum length of sequence
        """
        super().__init__()
        self.embedding = Embedding(vocab_size, dim_embedding)
        self.positional_encoding = PositionalEncoding(max_sequence_len, dim_embedding)
        self.decoder = Decoder(heads, dim_embedding, Nx, prob_dropout)
        self.linear = Linear(dim_embedding, vocab_size)
        self.softmax = Softmax()
    
    def forward(self, x, encoder_output, mask=None):
        """
        Args:
            x: input vector
            encoder_output: output vector from encoder
            mask: mask matrix (optional)
        
        Returns:
            output vector from decoder
        """
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.decoder(x, encoder_output, mask)
        x = self.linear(x)
        x = self.softmax(x)
        return x


Y por último escribimos la clase transformer

In [11]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, dim_embedding, max_sequence_len, heads, Nx, prob_dropout=0.1):
        """
        Args:
            vocab_size: size of vocabulary
            dim_embedding: dimension of embedding vector
            max_sequence_len: maximum length of sequence
            heads: number of heads
            Nx: number of decoder layers
        """
        super().__init__()
        self.encoder = TransformerEncoder(vocab_size, dim_embedding, max_sequence_len, heads, Nx, prob_dropout)
        self.decoder = TransformerDecoder(heads, dim_embedding, Nx, vocab_size, max_sequence_len, prob_dropout)
    
    def forward(self, source, target, mask=None):
        """
        Args:
            source: source vector
            target: target vector

        Returns:
            output vector from decoder
        """
        encoder_output = self.encoder(source)
        decoder_output = self.decoder(target, encoder_output, mask)
        return decoder_output


Rescatamos las función que crea la máscara para el transformer

In [12]:
def create_mask(sequence_len):
    """
    Args:
        sequence_len: length of sequence
        
    Returns:
        mask matrix
    """
    mask = torch.tril(torch.ones((sequence_len, sequence_len)))
    return mask

Volvemos a definir la función que obtiene el embbeding más el positional encoding de BERT

In [5]:
import torch
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

Creamos una sentencia para el encoder y otra para el decoder

In [6]:
sentence_encoder = "I gave the dog a bone because it was hungry"
sentence_decoder = "Le di un hueso al perro porque tenía hambre"

tokens_encoder, input_ids_encoder, token_embeddings_encoder, positional_encodings_encoder, embeddings_with_positional_encoding_encoder = extract_embeddings(sentence_encoder)
tokens_decoder, input_ids_decoder, token_embeddings_decoder, positional_encodings_decoder, embeddings_with_positional_encoding_decoder = extract_embeddings(sentence_decoder)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predic

Creamos un objeto `transformer` y obtenemos su salida

In [13]:
vocab_size = 30522
dim_embedding = max(token_embeddings_encoder.shape[-1], token_embeddings_decoder.shape[-1])
max_sequence_len = max(token_embeddings_encoder.shape[1], token_embeddings_decoder.shape[1])
heads = 8
Nx = 6
prob_dropout = 0.1
print(f"vocab_size: {vocab_size}, dim_embedding: {dim_embedding}, max_sequence_len: {max_sequence_len}, heads: {heads}, Nx: {Nx}, prob_dropout: {prob_dropout}")
transformer = Transformer(vocab_size, dim_embedding, max_sequence_len, heads, Nx, prob_dropout)

vocab_size: 30522, dim_embedding: 768, max_sequence_len: 16, heads: 8, Nx: 6, prob_dropout: 0.1


In [14]:
mask = create_mask(max_sequence_len)
mask.shape

torch.Size([16, 16])

In [15]:
transformer_output = transformer(input_ids_encoder, input_ids_decoder, mask=mask)
transformer_output.shape

torch.Size([1, 16, 30522])

Al igual que en el notebook anterior obtenemos 1x16x30522, es decir 1 batch, 16 tokens y 30522 posibles tokens del vocabulario